In [1]:
import IPython.display as ipd

import numpy as np
import torch


# import stft_64_pad_0 as stft
import stft_64 as stft
from audio_processing import griffin_lim

from scipy.io.wavfile import read
import time

In [2]:
N = 4

In [3]:
all_4_pair = {}
for i in range(N):
    for j in range(N):
        for k in range(N):
            for l in range(N):
                temp = [i,j,k,l]
                temp.sort()
                if tuple(temp) not in all_4_pair:
                    all_4_pair[tuple(temp)] = 0
                    
print(len(all_4_pair))
# print(all_4_pair)

all_4_arr = list(all_4_pair)
# print(all_4_arr)

35


In [4]:
def multiple_two_dict(dic1, dic2):
    new_dic = all_4_pair.copy()
    for x1 in dic1:
        for x2 in dic2:
            temp = list(x1 + x2)
            temp.sort()
            temp = tuple(temp)
            if temp not in new_dic:
                new_dic[temp] = 0
            new_dic[temp] += (dic1[x1]*dic2[x2])
    return new_dic

# multiple_two_dict({(0,1):1}, {(0,2):1})

In [5]:
dic_arr = []
for k in range(N//2):
    dic_arr.append({})
    for i in range(N):
        i_plus_k = (i+k)%N
        temp =[i, i_plus_k]
        temp.sort()
        temp = tuple(temp)
        dic_arr[-1][temp] = 1
        
dic_arr.append({})
for i in range(N//2):
    i_plus_k = (i+N//2)%N
    temp =[i, i_plus_k]
    temp.sort()
    temp = tuple(temp)
    dic_arr[-1][temp] = 1

# for k in range(N//2+1):
#     print(dic_arr[k])

In [23]:
dic_arr_4 = []
for i in range(N//2+1):
    for j in range(i+1):
        dic_arr_4.append(multiple_two_dict(dic_arr[i],dic_arr[j]))
print(len(dic_arr_4))
print(len(dic_arr_4[0]))
print(dic_arr_4[0])

6
35
{(0, 0, 0, 0): 1, (0, 0, 0, 1): 0, (0, 0, 0, 2): 0, (0, 0, 0, 3): 0, (0, 0, 1, 1): 2, (0, 0, 1, 2): 0, (0, 0, 1, 3): 0, (0, 0, 2, 2): 2, (0, 0, 2, 3): 0, (0, 0, 3, 3): 2, (0, 1, 1, 1): 0, (0, 1, 1, 2): 0, (0, 1, 1, 3): 0, (0, 1, 2, 2): 0, (0, 1, 2, 3): 0, (0, 1, 3, 3): 0, (0, 2, 2, 2): 0, (0, 2, 2, 3): 0, (0, 2, 3, 3): 0, (0, 3, 3, 3): 0, (1, 1, 1, 1): 1, (1, 1, 1, 2): 0, (1, 1, 1, 3): 0, (1, 1, 2, 2): 2, (1, 1, 2, 3): 0, (1, 1, 3, 3): 2, (1, 2, 2, 2): 0, (1, 2, 2, 3): 0, (1, 2, 3, 3): 0, (1, 3, 3, 3): 0, (2, 2, 2, 2): 1, (2, 2, 2, 3): 0, (2, 2, 3, 3): 2, (2, 3, 3, 3): 0, (3, 3, 3, 3): 1}


In [48]:



target = all_4_pair.copy()
target_arr = [[0,1,2,3],
             ]

for s1 in range(1):
    for i in target_arr[s1]:
        for j in target_arr[s1]:
            for k in target_arr[s1]:
                for l in target_arr[s1]:
                    temp = [i,j,k,l]
                    temp.sort()
                    # target[tuple(temp)] += (((1+np.sqrt(3)*1j)/np.sqrt(4))**(i+j+k+l))
                    target[tuple(temp)] += ((1j)**(i+j+k+l))
print(target)

{(0, 0, 0, 0): (1+0j), (0, 0, 0, 1): 4j, (0, 0, 0, 2): (-4+0j), (0, 0, 0, 3): -4j, (0, 0, 1, 1): (-6+0j), (0, 0, 1, 2): -12j, (0, 0, 1, 3): (12+0j), (0, 0, 2, 2): (6+0j), (0, 0, 2, 3): 12j, (0, 0, 3, 3): (-6+0j), (0, 1, 1, 1): -4j, (0, 1, 1, 2): (12+0j), (0, 1, 1, 3): 12j, (0, 1, 2, 2): 12j, (0, 1, 2, 3): (-24+0j), (0, 1, 3, 3): -12j, (0, 2, 2, 2): (-4+0j), (0, 2, 2, 3): -12j, (0, 2, 3, 3): (12+0j), (0, 3, 3, 3): 4j, (1, 1, 1, 1): (1+0j), (1, 1, 1, 2): 4j, (1, 1, 1, 3): (-4+0j), (1, 1, 2, 2): (-6+0j), (1, 1, 2, 3): -12j, (1, 1, 3, 3): (6+0j), (1, 2, 2, 2): -4j, (1, 2, 2, 3): (12+0j), (1, 2, 3, 3): 12j, (1, 3, 3, 3): (-4+0j), (2, 2, 2, 2): (1+0j), (2, 2, 2, 3): 4j, (2, 2, 3, 3): (-6+0j), (2, 3, 3, 3): -4j, (3, 3, 3, 3): (1+0j)}


In [49]:
A = np.zeros((len(all_4_pair), len(dic_arr_4)), dtype=np.complex128)
b = np.zeros((len(all_4_pair),1), dtype=np.complex128)

for i in range(len(all_4_arr)):
    b[i,0] = target[all_4_arr[i]]
# for i in range(len(all_4_arr)):
#     b[i,0] = dic_arr_4[0][all_4_arr[i]]+ 2*dic_arr_4[2][all_4_arr[i]]
print(np.sum(b>0))
    
for k in range(len(dic_arr_4)):
    for i in range(len(all_4_arr)):
        A[i,k] = dic_arr_4[k][all_4_arr[i]]

18


In [50]:
Q, R = np.linalg.qr(A, mode='reduced')
br = np.real(b)
bi = np.imag(b)

Qbr = np.matmul(Q.T, br)
ansr = np.linalg.solve(R,Qbr)
Qbi = np.matmul(Q.T, bi)
ansi = np.linalg.solve(R,Qbi)

ans = ansr + 1.j*ansi

b_hat = np.matmul(A, ans)
print(np.sum(np.abs(b_hat-b)))
print(A.shape)
print(ans.T)
# print(b.T)

256.0
(35, 6)
[[ 2.09876543e-01+5.61418815e-17j  3.58551055e-16-5.55111512e-17j
  -4.93827160e-01+2.24567526e-16j  4.49382716e+00-2.24567526e-16j
  -7.17102110e-16+4.44089210e-16j -5.48148148e+00-8.42128222e-16j]]
